In [1]:
!pip install --upgrade langchain==0.1.6
!pip install --upgrade langchain-community==0.0.19
!pip install --upgrade langchain-core==0.1.23
!pip install --upgrade faiss-cpu
!pip install --upgrade sentence-transformers
!pip install --upgrade langchain_aws

  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import userdata
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ["AWS_DEFAULT_REGION"] = userdata.get('AWS_DEFAULT_REGION')

In [32]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain_aws import ChatBedrock
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import PromptTemplate

file_path = '/content/faq/it_sector.txt'
with open(file_path, 'r', encoding='utf-8') as file:
      text = file.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_text(text)

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0"
)

vectorstore = FAISS.from_texts(texts=docs, embedding=embeddings)

llm = ChatBedrock(
   model_id="mistral.mistral-large-2402-v1:0",
   temperature=0.5
)

template = """
You are an IT support chatbot. If the user greets you, respond politely and let them know you are here to assist with IT support questions.
If they ask a question about the IT sector, use the following context to answer the question.
If the question is out of context, respond politely that you are an IT support chatbot and can only assist with IT-related questions.
If you don't know the answer, just say you don't know.
You should respond with short and concise answers, no longer than 2 sentences.

Context: {context}
Question: {question}
Answer:
"""

prompt = PromptTemplate(template=template, input_variables=["context", "question"])

rag_chain = (
  {"context": vectorstore.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [33]:
# Function for generating LLM response
def generate_response(input):
    result = rag_chain.invoke(input)
    return result

In [36]:
input = "What are some tips for speeding up my slow laptop?"

In [37]:
response = generate_response(input)
print(response)

 Check for background processes consuming resources, update your operating system and drivers, remove unnecessary startup programs, and perform disk cleanup. Consider upgrading hardware if necessary.
